In [1]:
import os
import sys

# Init project path
PROJECT_DIR = os.getcwd() + "/../"
sys.path.insert(0, PROJECT_DIR)
%load_ext autoreload
%autoreload 2

In [2]:
import json

config: dict = json.load(open("./../configs/lightgbm_0.json"))

In [3]:
from db import table_load

# Load data
train = table_load(
    table_name=config["dataset"]["train_table"],
    cols=config["features"]["id"]
    + config["features"]["target"]
    + config["features"]["train"],
)
test = table_load(
    table_name=config["dataset"]["test_table"],
    cols=config["features"]["id"] + config["features"]["train"],
)

[table load (train)] start
[table load (train)] done in 0.0140 s

[table load (test)] start
[table load (test)] done in 0.0084 s



In [5]:
from models import LightGBM, Model
models: dict = {"lightgbm": LightGBM}
model: Model = models[config["model"]["name"]]()
# Cross Validation
cv_models, result = model.cross_validatoin(
    n_splits=config["cv"]["n_splits"],
    random_state=config["cv"]["random_state"],
    train=train,
    target_cols=config["features"]["target"],
    train_cols=config["features"]["train"],
    categorical_cols=config["features"]["categorical"],
    params=config["model"],
)

[CV No.0] start
Accuracy: 0.6703910614525139
[CV No.0] done in 0.2513 s

[CV No.1] start
Accuracy: 0.6927374301675978
[CV No.1] done in 0.2441 s

[CV No.2] start
Accuracy: 0.7191011235955056
[CV No.2] done in 0.2380 s

[CV No.3] start
Accuracy: 0.6797752808988764
[CV No.3] done in 0.2563 s

[CV No.4] start
Accuracy: 0.6271186440677966
[CV No.4] done in 0.2555 s

